# Face Area Calculations

## Overview

This notebook will showcase the different area calculation options provided by UXarray

For more details on how to load in data, check out our [previous usage
example](https://uxarray.readthedocs.io/en/latest/examples/001-read-grid-data.html)

**This notebook has the following sections:**
1. Compute total area of all faces
2. Options for accuracy of area calculation functions
3. Getting area of all the faces (not total)
4. Calculate area of a single triangle
5. Calculate area of multiple vertices in spherical coordinates


We will be using the `outCSne30.ug` grid file which is encoded in the UGRID convention.
Let us first read in the data:

In [20]:
import uxarray as ux
import numpy as np
import xarray as xr

In [21]:
# Base data path
base_path = "../../test/meshfiles/"

# Path to Grid files
ugrid_path = base_path + "/ugrid/outCSne30/outCSne30.ug"

# Load grid files and create UXarray Grid objects
ugrid_ds = xr.open_dataset(ugrid_path)

ugrid = ux.Grid(ugrid_ds)
ugrid.ds

<xarray.Dataset>
Dimensions:               (nMesh2_face: 5400, nMaxMesh2_face_nodes: 4,
                           nMesh2_node: 5402)
Coordinates:
    Mesh2_node_x          (nMesh2_node) float64 ...
    Mesh2_node_y          (nMesh2_node) float64 ...
Dimensions without coordinates: nMesh2_face, nMaxMesh2_face_nodes, nMesh2_node
Data variables:
    Mesh2                 int32 ...
    Mesh2_face_nodes      (nMesh2_face, nMaxMesh2_face_nodes) int64 0 8 ... 298
    Mesh2_face_dimension  (nMesh2_face) int64 4 4 4 4 4 4 4 4 ... 4 4 4 4 4 4 4

## 1. Calculate total area of all faces
By calling the function `calculate_total_face_area()`

In [22]:
# Calculate total area of the first (ugrid) mesh
area = ugrid.calculate_total_face_area()
area
# 4*Pi is the area 4*3.14 = 12.56

12.566370614678554

## 2. Options for `Grid.calculate_total_face_area` functions

Quadrature Rule: gaussian or triangular

Order: 

       1 to 10              for Gaussian 

       1, 4, 8, 10 and 12   for triangular

Default: Triangular and order 4

In [23]:

ugrid.calculate_total_face_area(quadrature_rule="triangular", order=1)

12.571403993719983

For the above result: notice that the area is different from the first mesh above

Now we use "triangular" quadrature rule and order 1

This is less time consuming as the order is only 1

## 3. Getting area of all the faces (not total)

We use the Grid property `face_area` to get the area of all the faces in the mesh. 

Calling `face_area` or `calculate_total_face_areas` again will use the cached value.


In [24]:
ugrid.face_areas

array([0.00211238, 0.00211285, 0.00210788, ..., 0.00210788, 0.00211285,
       0.00211238])

Now calculate the area again with `compute face_areas` function
pass arguments: quadrature_rule "gaussian" and order 4

In [25]:
all_face_areas = ugrid.compute_face_areas(quadrature_rule="gaussian", order=4)
all_face_areas.sum()

12.566370614359112

## 4. Calculate area of a single triangle (cartesian coordinates)

For this section, we define a new 1 triangle face grid with 3 vertices

Assume the units in meters - this is a big triangle!

In [26]:
verts = [[[0.57735027, -5.77350269e-01, -0.57735027],
         [0.57735027, 5.77350269e-01, -0.57735027],
         [-0.57735027, 5.77350269e-01, -0.57735027]]]
# Note: initialization using array of vertices

# load our vertices into a UXarray Grid object
vgrid = ux.Grid(verts, vertices=True,
                        islatlon=False,
                        concave=False)

# Note the incorrect units for the x and y coordinates (expand the cell Mesh2_node_x etc.
# See units are set as degree_east and degree_north)
vgrid.ds

<xarray.Dataset>
Dimensions:               (nMesh2_node: 3, nMesh2_face: 1,
                           nMaxMesh2_face_nodes: 3)
Dimensions without coordinates: nMesh2_node, nMesh2_face, nMaxMesh2_face_nodes
Data variables:
    Mesh2                 float64 nan
    Mesh2_node_x          (nMesh2_node) float64 -0.5774 0.5774 0.5774
    Mesh2_node_y          (nMesh2_node) float64 0.5774 -0.5774 0.5774
    Mesh2_node_z          (nMesh2_node) float64 -0.5774 -0.5774 -0.5774
    Mesh2_face_nodes      (nMesh2_face, nMaxMesh2_face_nodes) int64 1 2 0
    Mesh2_face_dimension  (nMesh2_face) int64 3

In [27]:
vgrid.calculate_total_face_area()

0.00020308171629085178

The above result (0.0002) is WRONG as the units are assumed to be spherical, when the coordinates are in cartesian system

In [28]:
# Set correct units for the x and y coordinates
vgrid.Mesh2_node_x.attrs["units"] = "m"
vgrid.Mesh2_node_y.attrs["units"] = "m"
vgrid.Mesh2_node_z.attrs["units"] = "m"

# It is important to set the units for the x and y coordinates
# otherwise the area will be incorrect, as the units are assumed to be spherical by default
# Without the above setting the coordinates will be assumed to be in spherical coordinates
# and the area will be calculated incorrectly

# Calculate the area of the triangle
area_gaussian = vgrid.calculate_total_face_area(
            quadrature_rule="gaussian", order=5)
area_gaussian

1.0475702709086985

## 5. Calculate area of multiple vertices in spherical coordinates

In [29]:
faces_verts_ndarray = np.array([
    np.array([[150, 10, 0], [160, 20, 0], [150, 30, 0], [135, 30, 0], [125, 20, 0],
              [135, 10, 0]]),
    np.array([[125, 20, 0], [135, 30, 0], [125, 60, 0], [110, 60, 0], [100, 30, 0],
              [105, 20, 0]]),
    np.array([[95, 10, 0], [105, 20, 0], [100, 30, 0], [85, 30, 0], [75, 20, 0],
              [85, 10, 0]]),
])

In [30]:
# load our vertices into a UXarray Grid object
verts_grid = ux.Grid(faces_verts_ndarray, vertices=True,
                        islatlon=True,
                        concave=False)
verts_grid.ds

<xarray.Dataset>
Dimensions:               (nMesh2_node: 14, nMesh2_face: 3,
                           nMaxMesh2_face_nodes: 6)
Dimensions without coordinates: nMesh2_node, nMesh2_face, nMaxMesh2_face_nodes
Data variables:
    Mesh2                 float64 nan
    Mesh2_node_x          (nMesh2_node) int64 75 85 85 95 ... 135 150 150 160
    Mesh2_node_y          (nMesh2_node) int64 20 10 30 10 30 ... 10 30 10 30 20
    Mesh2_node_z          (nMesh2_node) int64 0 0 0 0 0 0 0 0 0 0 0 0 0 0
    Mesh2_face_nodes      (nMesh2_face, nMaxMesh2_face_nodes) int64 11 13 ... 1
    Mesh2_face_dimension  (nMesh2_face) int64 6 6 6

In [31]:
verts_grid.face_areas

array([0.14323746, 0.25118746, 0.12141312])

In [32]:
cart_x = [
   0.577340924821405, 0.577340924821405, 0.577340924821405,
   0.577340924821405
]
cart_y = [
   0.577343045516932, 0.577343045516932, -0.577343045516932,
   -0.577343045516932
]
cart_z = [
   0.577366836872017, -0.577366836872017, 0.577366836872017,
   -0.577366836872017
]


In [33]:
ux.helpers.calculate_face_area(np.array(cart_x), np.array(cart_y), np.array(cart_z))

/opt/homebrew/anaconda3/envs/uxarray/lib/python3.10/site-packages/numba/core/ir_utils.py:2147: NumbaPendingDeprecationWarning: 
Encountered the use of a type that is scheduled for deprecation: type 'reflected list' found for argument 'x' of function 'calculate_face_area'.

For more information visit https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-reflection-for-list-and-set-types

File "../../../../../opt/homebrew/anaconda3/envs/uxarray/lib/python3.10/site-packages/uxarray/helpers.py", line 101:
@njit
def calculate_face_area(x,
^

  warnings.warn(NumbaPendingDeprecationWarning(msg, loc=loc))
/opt/homebrew/anaconda3/envs/uxarray/lib/python3.10/site-packages/numba/core/ir_utils.py:2147: NumbaPendingDeprecationWarning: 
Encountered the use of a type that is scheduled for deprecation: type 'reflected list' found for argument 'y' of function 'calculate_face_area'.

For more information visit https://numba.readthedocs.io/en/stable/reference/deprecation.html#d

1.1093057890529491e-21